In [ ]:
class ImageProcessorFactory:
    def __init__(self, pixels, width, height):
        self.pixels = pixels
        self.width = width
        self.height = height

    def get_pixel(self, x, y):
        return self.pixels[y * self.width + x]

    def set_pixel(self, x, y, pixel):
        self.pixels[y * self.width + x] = pixel

    def to_grayscale(self):
        for i in range(len(self.pixels)):
            r, g, b = self.pixels[i]
            gray = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            self.pixels[i] = (gray, gray, gray)
        return self

    def apply_blur(self):
        new_pixels = [(0, 0, 0)] * len(self.pixels)

        for y in range(1, self.height - 1):
            for x in range(1, self.width - 1):
                r_sum = g_sum = b_sum = 0
                count = 0

                for j in range(-1, 2):
                    for i in range(-1, 2):
                        r, g, b = self.get_pixel(x + i, y + j)
                        r_sum += r
                        g_sum += g
                        b_sum += b
                        count += 1

                new_pixel = (r_sum // count, g_sum // count, b_sum // count)
                self.set_pixel(x, y, new_pixel)

        self.pixels = new_pixels
        return self

    def invert_colors(self):
        for i in range(len(self.pixels)):
            r, g, b = self.pixels[i]
            self.pixels[i] = (255 - r, 255 - g, 255 - b)
        return self

    def get_pixels(self):
        return self.pixels

In [ ]:
# Read the image data
input_image_path = "input_image.bmp"
header, data, width, height, bits_per_pixel = read_image(input_image_path)

# Create the factory object with the pixel data
image_processor = ImageProcessorFactory(pixels, width, height)

# Apply desired effects
image_processor.to_grayscale()
image_processor.apply_blur()
image_processor.invert_colors()

# Get the modified pixel data
enhanced_pixels = image_processor.get_pixels()

# Convert the pixel data to bytes
enhanced_data = b''
for pixel in enhanced_pixels:
    enhanced_data += bytes(pixel)

# Save the enhanced image to a new file
output_image_path = "output_image.bmp"
write_image(output_image_path, header, enhanced_data)

In [ ]:
#Updated Read
import struct
import numpy as np

def custom_imread(filepath):
    try:
        with open(filepath, 'rb') as f:
            header = f.read(54)
            width, height, bits_per_pixel = struct.unpack("<3I", header[18:30])
            data = f.read()
    except IOError:
        print(f"Error: Unable to open file '{filepath}'.")
        return None
    except struct.error:
        print("Error: Invalid BMP header.")
        return None

    # Calculate the padding for each row
    padding = (4 - (width * 3) % 4) % 4

    try:
        image_array = np.zeros((height, width, 3), dtype=np.uint8)

        # Parse the image data into a NumPy array
        index = 0
        for y in range(height - 1, -1, -1):
            for x in range(width):
                b = data[index]
                g = data[index + 1]
                r = data[index + 2]
                image_array[y, x] = [b, g, r]
                index += 3
            index += padding
    except IndexError:
        print("Error: Invalid BMP data.")
        return None

    return image_array

# Read an image using the custom_imread function
input_image_path = "input_image.bmp"
image = custom_imread(input_image_path)

if image is not None:
    # Check the data type and shape of the image
    print("Data type:", type(image))  # numpy.ndarray
    print("Shape:", image.shape)      # (height, width, num_channels)
else:
    print("Failed to load image.")

In [ ]:
import struct
import numpy as np

def custom_imwrite(filepath, image):
    height, width, _ = image.shape

    # Calculate the padding for each row
    padding = (4 - (width * 3) % 4) % 4

    # Write the BMP header
    header_size = 54
    image_size = (width * 3 + padding) * height
    file_size = header_size + image_size
    header = struct.pack(
        "<2sI4xI4xIIIIHHIIIIII",
        b"BM", file_size, header_size, 40, width, height, 1, 24, 0, image_size, 0, 0, 0, 0
    )

    # Write the image data
    data = b""
    for y in range(height - 1, -1, -1):
        for x in range(width):
            b, g, r = image[y, x]
            data += struct.pack("BBB", b, g, r)
        data += b"\x00" * padding

    try:
        with open(filepath, "wb") as f:
            f.write(header)
            f.write(data)
        return True
    except IOError:
        print(f"Error: Unable to save file '{filepath}'.")
        return False

# Save an image using the custom_imwrite function
output_image_path = "output_image.bmp"

if custom_imwrite(output_image_path, image):
    print("Image saved successfully.")
else:
    print("Failed to save image.")

In [ ]:
from abc import ABC, abstractmethod
import numpy as np

class AbstractImageHandler(ABC):
    @abstractmethod
    def read_image(self, filepath: str) -> np.ndarray:
        pass

    @abstractmethod
    def write_image(self, filepath: str, image: np.ndarray) -> bool:
        pass

In [ ]:
import struct

class BMPHandler(AbstractImageHandler):
    def read_image(self, filepath: str) -> np.ndarray:
        # Custom read_image implementation for BMP files
        # (the same as the custom_imread function provided earlier)
        pass

    def write_image(self, filepath: str, image: np.ndarray) -> bool:
        # Custom write_image implementation for BMP files
        # (the same as the custom_imwrite function provided earlier)
        pass

In [ ]:
class ImageHandlerFactory:
    def get_handler(self, file_extension: str) -> AbstractImageHandler:
        if file_extension.lower() == ".bmp":
            return BMPHandler()
        else:
            raise ValueError(f"Unsupported file format: {file_extension}")

In [ ]:
input_image_path = "input_image.bmp"
output_image_path = "output_image.bmp"

# Get the file extension
input_extension = input_image_path[input_image_path.rfind("."):]

# Get the appropriate image handler using the factory
factory = ImageHandlerFactory()
handler = factory.get_handler(input_extension)

# Read the image
image = handler.read_image(input_image_path)

# Process the image (e.g., convert to grayscale)
# ... (your image processing code here)

# Get the file extension for the output image
output_extension = output_image_path[output_image_path.rfind("."):]

# Get the appropriate image handler for the output file
handler = factory.get_handler(output_extension)

# Write the image
handler.write_image(output_image_path, image)

In [ ]:
from abc import ABC, abstractmethod
import numpy as np
import struct
import os

class AbstractImageHandler(ABC):
    @abstractmethod
    def read_image(self, filepath: str) -> np.ndarray:
        pass

    @abstractmethod
    def write_image(self, filepath: str, image: np.ndarray) -> bool:
        pass


class BMPHandler(AbstractImageHandler):
    def read_image(self, filepath: str) -> np.ndarray:
        # Custom read_image implementation for BMP files
        # (the same as the custom_imread function provided earlier)
        pass

    def write_image(self, filepath: str, image: np.ndarray) -> bool:
        # Custom write_image implementation for BMP files
        # (the same as the custom_imwrite function provided earlier)
        pass


class ImageHandlerFactory:
    handlers = {
        ".bmp": BMPHandler,
    }

    @staticmethod
    def get_handler(filepath: str) -> AbstractImageHandler:
        file_extension = os.path.splitext(filepath)[1].lower()
        if file_extension in ImageHandlerFactory.handlers:
            return ImageHandlerFactory.handlers[file_extension]()
        else:
            raise ValueError(f"Unsupported file format: {file_extension}")


# Usage example
input_image_path = "input_image.bmp"
output_image_path = "output_image.bmp"

# Get the appropriate image handler using the factory
handler = ImageHandlerFactory.get_handler(input_image_path)

# Read the image
image = handler.read_image(input_image_path)

# Process the image (e.g., convert to grayscale)
# ... (your image processing code here)

# Get the appropriate image handler for the output file
handler = ImageHandlerFactory.get_handler(output_image_path)

# Write the image
handler.write_image(output_image_path, image)